In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

In [2]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import pandas as pd

In [3]:
physical_devices =  tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32")/ 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

How to save and load model weights
Save and load entire model (Serializing model)
 - Save Weights
 - Model Architecture
 - Training Configuration (model.compile())
 - Optimizer and states

In [13]:
#CNN -> BatchNorm -> ReLU (Common structure)
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding = 'same')
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        #print(x.shape)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

In [14]:
model = keras.Sequential(
    [
        CNNBlock(32),
        CNNBlock(64),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10),
    ]
)

In [18]:
#load model
#model.load_weights('saved_model/')
model = keras.models.load_model('saved_model/')

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [20]:

model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)
print(model.summary())
model.evaluate(x_test, y_test, batch_size=64, verbose=2)


938/938 - 36s - loss: 0.0313 - accuracy: 0.9901 - 36s/epoch - 39ms/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_block_6 (CNNBlock)      (None, 28, 28, 32)        448       
                                                                 
 cnn_block_7 (CNNBlock)      (None, 28, 28, 64)        18752     
                                                                 
 cnn_block_8 (CNNBlock)      (None, 28, 28, 128)       74368     
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 10)                1003530   
                                                                 
Total params: 1,097,098
Trainable params: 1,096,650
Non-trainable params: 448
____________________________________

[0.041485272347927094, 0.9872999787330627]

In [17]:
#save model
model.save_weights('saved_model/')
model.save('saved_model/')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets
